In [1]:
!pip install --upgrade sentence-transformers
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.2.1
    Uninstalling sentence-transformers-3.2.1:
      Successfully uninstalled sentence-transformers-3.2.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of 

In [2]:
import torch

def check_gpu():
    """
    Verifica se a GPU está disponível.
    Lança um erro caso a GPU não esteja ativa.
    """
    if not torch.cuda.is_available():
        raise EnvironmentError("Nenhuma GPU foi detectada! Certifique-se de que ativou a GPU no ambiente de execução.")
    else:
        print(f"GPU detectada: {torch.cuda.get_device_name(0)}")

# Exemplo de uso
try:
    check_gpu()
except EnvironmentError as e:
    print(e)


GPU detectada: Tesla T4


In [3]:
import json

# Function to preprocess a single issue
def preprocess_issue(issue):
    title = issue.get("title", "").strip()
    body = issue.get("body", "").strip()
    comments = " ".join([comment.get("comment_body", "").strip() for comment in issue.get("comments", [])])
    return f"{title} [SEP] {body} [SEP] {comments}"

# Function to load issues from a JSON file
def load_json(file_path):
    with open(file_path, "r") as f:
        return json.load(f)

# Load issues from the JSON file
file_path = "refined_issues.json"  # Replace with your file path
issues_data = load_json(file_path)

# Preprocess all issues
issues = [preprocess_issue(issue) for issue in issues_data]

# Print the first 3 preprocessed issues for verification
print("\n".join(issues[:3]))

update psw development image [SEP] <MENTION> March <NUMBER> <CODE> <NUMBER>49 blocks <ISSUE_REF> laptop <CODE_BLOCK> cc <MENTION> update ucode laptop Copied original issue sunblazeucbekiden<ISSUEREF>_ [SEP]    
Support IPFS checkpointing state [SEP]  [SEP]  
Compute node rewards gas model [SEP] need way measure gas consumption request reward compute nodes Compute nodes may specify policies [SEP]  


In [4]:
import random

# Function to create random augmentations
def augment_text(text):
    # Example augmentations: shuffle sentences, remove random words
    sentences = text.split(" [SEP] ")
    random.shuffle(sentences)
    return " [SEP] ".join(sentences)

# Create InputExample pairs
from sentence_transformers import InputExample

train_examples = [
    InputExample(texts=[issue, augment_text(issue)])  # Positive pair
    for issue in issues
]

In [5]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)

In [6]:
from sentence_transformers import SentenceTransformer, losses

# Define the model and loss
model = SentenceTransformer("all-MiniLM-L6-v2")
train_loss = losses.MultipleNegativesRankingLoss(model)

# Train the model
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=4,
    warmup_steps=100,
    output_path="./output/sbert_finetuned_issues"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.000400
1000,0.000000
1500,0.000000
2000,0.000000
2500,0.000000
3000,0.000100
3500,0.000100
4000,0.000000
4500,0.000000
5000,0.000000


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

In [7]:
import shutil

# Define the directory to zip
model_dir = "./output/sbert_finetuned_issues"
zip_file = "./sbert_finetuned_issues.zip"

# Create a zip file
shutil.make_archive(model_dir, 'zip', model_dir)

print(f"Zipped model saved as {zip_file}")


Zipped model saved as ./sbert_finetuned_issues.zip
